# 基线模型

In [23]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from datetime import date, timedelta

In [24]:
DATA_DIR = "./data/"
df_train = pd.read_csv(
    DATA_DIR + "train.csv",
    parse_dates=['date'],
    converters={
        'onpromotion': bool,
        'store_nbr': str,
        'sales': lambda x: np.log1p(float(x)) if pd.notnull(x) else np.nan
    }
)
df_test = pd.read_csv(
    DATA_DIR + 'test.csv',
    parse_dates=['date'],
    converters={
        'onpromotion': bool,
        'store_nbr': str
    }
)
stores = pd.read_csv(
    DATA_DIR + 'stores.csv',
    converters={
        'store_nbr': str
    }
)

In [25]:
def view_df_base_info(df: pd.DataFrame):
    # 显示前几行数据
    display(df.head().style.set_caption("Preview of the DataFrame").set_table_attributes('style="font-size: 14px;"'))
    
    # 显示数据类型
    dtype_df = df.dtypes.reset_index()
    dtype_df.columns = ['Column', 'Data Type']
    display(dtype_df.style.set_caption("Data Types").set_table_attributes('style="font-size: 14px;"'))
    
    # 显示描述统计信息
    display(df.describe().style.set_caption("Descriptive Statistics").set_table_attributes('style="font-size: 14px;"'))
    
    # 显示缺失值比例
    null_mean = df.isnull().mean().reset_index()
    null_mean.columns = ['Column', 'Missing Value Ratio']
    display(null_mean.style.set_caption("Missing Value Ratio").set_table_attributes('style="font-size: 14px;"'))

In [26]:
view_df_base_info(df_train)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01 00:00:00,1,AUTOMOTIVE,0.000000,True
1,1,2013-01-01 00:00:00,1,BABY CARE,0.000000,True
2,2,2013-01-01 00:00:00,1,BEAUTY,0.000000,True
3,3,2013-01-01 00:00:00,1,BEVERAGES,0.000000,True
4,4,2013-01-01 00:00:00,1,BOOKS,0.000000,True


,Column,Data Type
0,id,int64
1,date,datetime64[ns]
2,store_nbr,object
3,family,object
4,sales,float64
5,onpromotion,bool


,id,date,sales
count,3000888.000000,3000888,3000888.000000
mean,1500443.500000,2015-04-24 08:27:04.703088384,2.926368
min,0.000000,2013-01-01 00:00:00,0.000000
25%,750221.750000,2014-02-26 18:00:00,0.000000
50%,1500443.500000,2015-04-24 12:00:00,2.484907
75%,2250665.250000,2016-06-19 06:00:00,5.282428
max,3000887.000000,2017-08-15 00:00:00,11.733810
std,866281.891642,nan,2.695122


,Column,Missing Value Ratio
0,id,0.000000
1,date,0.000000
2,store_nbr,0.000000
3,family,0.000000
4,sales,0.000000
5,onpromotion,0.000000


In [27]:
view_df_base_info(df_test)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16 00:00:00,1,AUTOMOTIVE,True
1,3000889,2017-08-16 00:00:00,1,BABY CARE,True
2,3000890,2017-08-16 00:00:00,1,BEAUTY,True
3,3000891,2017-08-16 00:00:00,1,BEVERAGES,True
4,3000892,2017-08-16 00:00:00,1,BOOKS,True


,Column,Data Type
0,id,int64
1,date,datetime64[ns]
2,store_nbr,object
3,family,object
4,onpromotion,bool


,id,date
count,28512.000000,28512
mean,3015143.500000,2017-08-23 12:00:00
min,3000888.000000,2017-08-16 00:00:00
25%,3008015.750000,2017-08-19 18:00:00
50%,3015143.500000,2017-08-23 12:00:00
75%,3022271.250000,2017-08-27 06:00:00
max,3029399.000000,2017-08-31 00:00:00
std,8230.849774,nan


,Column,Missing Value Ratio
0,id,0.000000
1,date,0.000000
2,store_nbr,0.000000
3,family,0.000000
4,onpromotion,0.000000


In [28]:
view_df_base_info(stores)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


,Column,Data Type
0,store_nbr,object
1,city,object
2,state,object
3,type,object
4,cluster,int64


,cluster
count,54.000000
mean,8.481481
std,4.693395
min,1.000000
25%,4.000000
50%,8.500000
75%,13.000000
max,17.000000


,Column,Missing Value Ratio
0,store_nbr,0.000000
1,city,0.000000
2,state,0.000000
3,type,0.000000
4,cluster,0.000000


In [30]:
df_train['store_nbr'].nunique(), df_train['family'].nunique(), df_train['store_nbr'].nunique() * df_train['family'].nunique() 

(54, 33, 1782)

## 数据准备

In [7]:
df_2017 = df_train.loc[df_train.date >= pd.to_datetime('2017-01-01')]
del df_train

In [8]:
promo_df_2017_train = (df_2017
    .set_index(['store_nbr', 'family', 'date'])
    [['onpromotion']]
    .unstack(level=-1)
    .fillna(False)
)
promo_df_2017_test = (df_test
    .set_index(['store_nbr', 'family', 'date'])
    [['onpromotion']]
    .unstack(level=-1)
    .fillna(False)
)

promo_df_2017_train.columns = promo_df_2017_train.columns.get_level_values(1)
promo_df_2017_test.columns = promo_df_2017_test.columns.get_level_values(1)

In [9]:
promo_df_2017_train.head()

date                  2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr family                              ...                           
1         AUTOMOTIVE        True        True  ...        True        True   
          BABY CARE         True        True  ...        True        True   
          BEAUTY            True        True  ...        True        True   
          BEVERAGES         True        True  ...        True        True   
          BOOKS             True        True  ...        True        True   

date                  2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr family                                                      
1         AUTOMOTIVE        True        True        True        True  
          BABY CARE         True        True        True        True  
          BEAUTY            True        True        True        True  
          BEVERAGES         True        True        True        True  
          BOOKS             True        True        True        True  

[5 rows x 227 columns]

In [10]:
promo_df_2017_test.head()

date                  2017-08-16  2017-08-17  2017-08-18  2017-08-19  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-08-20  2017-08-21  2017-08-22  2017-08-23  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr family                                                       
1         AUTOMOTIVE        True        True        True        True   
          BABY CARE         True        True        True        True   
          BEAUTY            True        True        True        True   
          BEVERAGES         True        True        True        True   
          BOOKS             True        True        True        True   

date                  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr family                                                      
1         AUTOMOTIVE        True        True        True        True  
          BABY CARE         True        True        True        True  
          BEAUTY            True        True        True        True  
          BEVERAGES         True        True        True        True  
          BOOKS             True        True        True        True

In [11]:
promo_2017 = pd.concat([promo_df_2017_train, promo_df_2017_test], axis=1)

df_2017 = (df_2017
    .set_index(['store_nbr', 'family', 'date'])
    [['sales']]
    .unstack(level=-1)
    .fillna(0)
)
df_2017.columns = df_2017.columns.get_level_values(1)
df_2017.head()

date                  2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr family                                                       
1         AUTOMOTIVE         0.0    1.791759    1.609438    0.693147   
          BABY CARE          0.0    0.000000    0.000000    0.000000   
          BEAUTY             0.0    0.000000    1.609438    1.609438   
          BEVERAGES          0.0    7.268920    8.033334    8.019613   
          BOOKS              0.0    0.000000    1.098612    1.098612   

date                  2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr family                                                       
1         AUTOMOTIVE    1.098612    1.098612    1.791759    0.000000   
          BABY CARE     0.000000    0.000000    0.000000    0.000000   
          BEAUTY        1.386294    2.397895    1.945910    0.693147   
          BEVERAGES     7.870166    7.923710    7.825645    7.213032   
          BOOKS         0.000000    0.000000    0.000000    0.693147   

date                  2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr family                              ...                           
1         AUTOMOTIVE    1.098612    1.386294  ...    1.945910    2.079442   
          BABY CARE     0.000000    0.000000  ...    0.000000    0.000000   
          BEAUTY        1.945910    2.397895  ...    1.098612    1.791759   
          BEVERAGES     8.318986    7.771067  ...    6.876265    7.643483   
          BOOKS         0.000000    1.386294  ...    0.000000    0.000000   

date                  2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr family                                                       
1         AUTOMOTIVE    1.609438    2.079442    2.302585    0.693147   
          BABY CARE     0.000000    0.000000    0.000000    0.000000   
          BEAUTY        1.098612    1.609438    2.397895    0.693147   
          BEVERAGES     7.791110    7.745868    7.774015    6.914731   
          BOOKS         0.000000    0.000000    0.000000    0.000000   

date                  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr family                                                      
1         AUTOMOTIVE    1.945910    0.693147    0.693147    1.609438  
          BABY CARE     0.000000    0.000000    0.000000    0.000000  
          BEAUTY        1.386294    0.693147    1.945910    1.609438  
          BEVERAGES     7.414573    6.689599    7.697121    7.571988  
          BOOKS         0.000000    0.000000    0.000000    0.000000  

[5 rows x 227 columns]

## 特征提取

In [12]:
def get_date_range(df, dt, forward_steps, periods, freq='D'):
    return df[pd.date_range(start=dt - timedelta(days=forward_steps), periods=periods, freq=freq)]

In [13]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_hist": get_date_range(df_2017, t2017, 1, 1).values.ravel(),
        "day_2_hist": get_date_range(df_2017, t2017, 2, 1).values.ravel(),
        "day_3_hist": get_date_range(df_2017, t2017, 3, 1).values.ravel(),
    })
    for i in (7, 14, 21, 30):
        # 窗口统计特征: 销量 diff/mean/meidan/max/min/std
        X[f'diff_{i}_day_mean'] = get_date_range(df_2017, t2017, i, i).diff(axis=1).mean(axis=1).values # type: ignore
        X[f'mean_{i}_day'] = get_date_range(df_2017, t2017, i, i).mean(axis=1).values # type: ignore
        X[f'median_{i}_day'] = get_date_range(df_2017, t2017, i, i).median(axis=1).values # type: ignore
        X[f'max_{i}_day'] = get_date_range(df_2017, t2017, i, i).max(axis=1).values # type: ignore
        X[f'min_{i}_day'] = get_date_range(df_2017, t2017, i, i).min(axis=1).values # type: ignore
        X[f'std_{i}_day'] = get_date_range(df_2017, t2017, i, i).std(axis=1).values # type: ignore

    for i in range(7):
        # 前4 10周每周的平均销量
        X[f'mean_4_dow{i}_2017'] = get_date_range(df_2017, t2017, 28 - i, 4, freq='7D').mean(axis=1).values # type: ignore
        X[f'mean_10_dow{i}_2017'] = get_date_range(df_2017, t2017, 70 - i, 10, freq='7D').mean(axis=1).values # type: ignore
    
    for i in range(16):
        # 未来16天是否是促销日
        X[f'promo_{i}'] = promo_2017[str(t2017 + timedelta(days=i))].values.astype(np.uint8)
    if is_train:
        y = df_2017[pd.date_range(t2017, periods=16)].values
        return X, y

    return X

In [21]:
df_2017.shape

(1782, 227)

In [19]:
prepare_dataset(date(2017, 7, 5))[0].shape

(1782, 57)

In [31]:
X_1, y_1 = [], []
t2017 = date(2017, 7, 5)

n_range = 14
for i in tqdm(range(n_range)):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 - delta)
    X_1.append(X_tmp)
    y_1.append(y_tmp)
X_train = pd.concat(X_1, axis=0)
y_train = np.concatenate(y_1, axis=0)

del X_1, y_1

100%|██████████| 14/14 [00:00<00:00, 16.02it/s]


In [32]:
# 验证集取 7 月 26 日 到 8 月 10 日 的数据
X_val, y_val = prepare_dataset(date(2017, 7, 26))
# 测试集取8月16日到8月31日的数据
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

## 模型训练

In [34]:
params = {
'num_leaves': 2**5 - 1,
'objective': 'regression_l2',
'max_depth': 8,
'min_data_in_leaf': 50,
'learning_rate': 0.05,
'feature_fraction': 0.75,
'bagging_fraction': 0.75,
'bagging_freq': 1,
'metric': 'l2',
'num_threads': 4
}
MAX_ROUNDS = 500
val_pred = []
test_pred = []
for i in range(16):
    print("====== Step %d ======" % (i+1))
    dtrain = lgb.Dataset(X_train, label=y_train[:, i])
    dval = lgb.Dataset(X_val, label=y_val[:, i], reference=dtrain)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval])
    
    val_pred.append(bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

====== Step 1 ======
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006283 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10455
[LightGBM] [Info] Number of data points in the train set: 24948, number of used features: 41
[LightGBM] [Info] Start training from score 3.518519
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli